In [ ]:
# !wget https://github.com/marcin119a/data/raw/refs/heads/main/data_gsn.zip
# !unzip data_gsn.zip &> /dev/null
# !rm data_gsn.zip

In [ ]:
import torch, torch.nn as nn
from torch.utils.data import Dataset, DataLoader, Subset
import pandas as pd
from PIL import Image
import numpy as np
import os
import matplotlib.pyplot as plt
from torchvision import transforms
import torch.nn.functional as F
from sklearn.metrics import f1_score, confusion_matrix
import pickle

torch.manual_seed(1)

In [ ]:
all_pairs = [(i, j) for i in range(6) for j in range(i + 1, 6)]
pair_to_idx = {p: k for k, p in enumerate(all_pairs)}

def class_id_to_pair_and_split(class_id: int):
    pair_idx = class_id // 9
    split_idx = class_id % 9
    ca = split_idx + 1
    cb = 10 - ca
    i, j = all_pairs[pair_idx]
    return (i, j), (ca, cb)

def class_id_to_pair(class_id: int):
    pair_idx = class_id // 9
    return all_pairs[pair_idx]

def counts_to_class_id(counts):
    if isinstance(counts, torch.Tensor):
        c = counts.detach().cpu().tolist()
    else:
        c = list(counts)

    nz = [i for i, v in enumerate(c) if v > 0]
  
    a, b = sorted(nz)
    ca = int(c[a])
    pair_index = pair_to_idx[(a, b)]

    class_id = pair_index * 9 + (ca - 1)
    return class_id

In [ ]:
class GSN(Dataset):
    def __init__(self, root, transform=None, transform_relabel=None):
        self.data_dir = os.path.join(root, "data")
        self.transform = transform
        self.transform_relabel = transform_relabel

        df = pd.read_csv(os.path.join(self.data_dir, "labels.csv"))
        self.names = df["name"].tolist()
        cols = ["squares", "circles", "up", "right", "down", "left"]
        self.labels = torch.tensor(df[cols].values, dtype=torch.float32)

    def __len__(self):
        return len(self.names)
    
    def __getitem__(self, index):
        name = self.names[index]
        img_path = os.path.join(self.data_dir, name)

        img = Image.open(img_path).convert("L")
        img = transforms.ToTensor()(img)
        
        if self.transform:
            img = self.transform(img)

        cnt = self.labels[index]

        if self.transform_relabel:
            img, cnt = self.transform_relabel(img, cnt)
        
        cls = counts_to_class_id(cnt)

        return img, cls, cnt

In [ ]:
class Augmentation:
    def __init__(self, p_hflip=0.5, p_vflip=0.5):
        self.p_hflip = p_hflip
        self.p_vflip = p_vflip

    def __call__(self, img, cnt):
        cnt = cnt.clone()

        k = torch.randint(0, 4, (1,)).item()
        if k > 0:
            img = torch.rot90(img, k=-k, dims=[1,2])
            dirs = cnt[2:6]
            dirs = torch.roll(dirs, shifts=k)
            cnt[2:6] = dirs

        if torch.rand(1).item() < self.p_hflip:
            img = torch.flip(img, dims=[2])
            cnt[[3, 5]] = cnt[[5, 3]]
        
        if torch.rand(1).item() < self.p_vflip:
            img = torch.flip(img, dims=[1])
            cnt[[2, 4]] = cnt[[4, 2]]

        return img, cnt

In [ ]:
class NeuralNetwork(nn.Module):
    def __init__(self, cls_hidden=256, cnt_hidden=256, dropout=0.3):
        super().__init__()

        self.backbone = nn.Sequential(
            nn.Conv2d(1, 8, 3, stride=1, padding=1), nn.ReLU(),     
            nn.Conv2d(8, 16, 3, stride=1, padding=1), nn.ReLU(),   
            nn.Conv2d(16, 32, 3, stride=1, padding=1), nn.ReLU(),
            nn.Conv2d(32, 64, 3, stride=1, padding=1), nn.ReLU(),
            nn.Flatten(start_dim=1),
            nn.Linear(64 * 28 * 28, 256), nn.ReLU()
        )

        self.head_cls = nn.Sequential(
            nn.Linear(256, cls_hidden),
            nn.ReLU(),
            nn.Dropout(p=dropout),
            nn.Linear(cls_hidden, 135),
            nn.LogSoftmax(dim=1)
        )

        self.head_cnt = nn.Sequential(
            nn.Linear(256, cnt_hidden),
            nn.ReLU(),
            nn.Linear(cnt_hidden, 6)
        )
    
    def forward(self, x):
        x = self.backbone(x)

        cls = self.head_cls(x)
        cnt = self.head_cnt(x)

        return cls, cnt

In [ ]:
def train_epoch(
    net: torch.nn.Module,
    device: torch.device,
    train_loader: torch.utils.data.DataLoader,
    optimizer: torch.optim.Optimizer,
    mode: str,
    lambda_cnt = None,
):
    net.train()
    total_loss = 0.0
    n_total = 0
    
    for img, cls_target, cnt_target in train_loader:
        img, cls_target, cnt_target = img.to(device), cls_target.long().to(device), cnt_target.to(device)

        optimizer.zero_grad()

        cls_pred, cnt_pred = net(img)
        
        cls_loss = F.nll_loss(cls_pred, cls_target)
        cnt_loss = F.smooth_l1_loss(cnt_pred, cnt_target)

        if mode == "cls_only":
            loss = cls_loss
        elif mode == "reg_only":
            loss = cnt_loss
        else:
            loss = cls_loss + lambda_cnt * cnt_loss

        loss.backward()
        optimizer.step()

        B = len(img)

        total_loss += loss.item() * B
        n_total += B

    epoch_loss = total_loss / n_total
    return epoch_loss


In [ ]:
def eval_epoch(
    net: torch.nn.Module,
    device: torch.device,
    test_loader: torch.utils.data.DataLoader,
    epoch: int,
    mode: str,
    lambda_cnt = None,
    verbose: bool = False,
):
    net.eval()
    total_loss = total_cls_loss = total_cnt_loss = 0.0
    n_total = 0
    correct = 0

    sum_sq_diff = 0.0
    n_cnt = 0

    with torch.no_grad():
        for img, cls_target, cnt_target in test_loader:
            img, cls_target, cnt_target = img.to(device), cls_target.long().to(device), cnt_target.to(device)

            cls_pred, cnt_pred = net(img)

            cls_loss = F.nll_loss(cls_pred, cls_target)
            cnt_loss = F.smooth_l1_loss(cnt_pred, cnt_target)

            if mode == "cls_only":
                loss = cls_loss
            elif mode == "reg_only":
                loss = cnt_loss
            else:
                loss = cls_loss + lambda_cnt * cnt_loss

            total_loss += loss.item() * len(img)
            total_cls_loss += cls_loss.item() * len(img)
            total_cnt_loss += cnt_loss.item() * len(img)
            n_total += len(img)

            pred = cls_pred.argmax(dim=1)
            correct += (pred == cls_target).sum().item()

            diff = cnt_pred - cnt_target
            sum_sq_diff += (diff ** 2).sum().item()
            n_cnt += diff.numel()

    epoch_loss = total_loss / n_total
    epoch_cls_loss = total_cls_loss / n_total
    epoch_cnt_loss = total_cnt_loss / n_total
    epoch_acc = correct / n_total

    epoch_rmse = (sum_sq_diff / n_cnt) ** 0.5

    if verbose:
        print(
            f"Eval Epoch: {epoch} | "
            f"acc: {epoch_acc:.4f} | "
            f"loss: {epoch_loss:.4f} | "
            f"cls_loss: {epoch_cls_loss:.4f} | "
            f"cnt_loss: {epoch_cnt_loss:.4f}"
        )
    return epoch_loss, epoch_acc, epoch_rmse

In [ ]:
def evaluate_metrics(net, device, loader):
    net.eval()

    all_cls_true = []
    all_cls_pred = []
    all_cnt_true = []
    all_cnt_pred = []

    with torch.no_grad():
        for img, cls_target, cnt_target in loader:
            img = img.to(device)

            cls_logits, cnt_pred = net(img)
            cls_pred = cls_logits.argmax(dim=1)

            all_cls_true.append(cls_target)
            all_cls_pred.append(cls_pred.to("cpu"))            
            all_cnt_true.append(cnt_target)
            all_cnt_pred.append(cnt_pred.to("cpu"))

    cls_true = torch.cat(all_cls_true)
    cls_pred = torch.cat(all_cls_pred)
    cnt_true = torch.cat(all_cnt_true)
    cnt_pred = torch.cat(all_cnt_pred)

    acc = (cls_true == cls_pred).float().mean().item()
    print(f"Top-1 accuracy: {acc}" )  

    cls_true_np = cls_true.numpy() 
    cls_pred_np = cls_pred.numpy() 
    macro_f1 = f1_score(cls_true_np, cls_pred_np, average="macro")
    print(f"Macro F1:{macro_f1}")

    correct_pair = 0
    total = len(cls_true)
    for t, p in zip(cls_true.tolist(), cls_pred.tolist()):
        if class_id_to_pair(int(t)) == class_id_to_pair(int(p)):
            correct_pair += 1
    pair_acc = correct_pair / total
    print(f"Per-pair accuracy: {pair_acc}")

    diff = cnt_pred - cnt_true
    mse_per_class = (diff ** 2).mean(dim=0)
    rmse_per_class = torch.sqrt(mse_per_class)
    mae_per_class = diff.abs().mean(dim=0)
    overall_rmse = torch.sqrt((diff ** 2).mean()).item()
    overall_mae = diff.abs().mean().item()  
    print(f"RMSE per class: {rmse_per_class.tolist()}")
    print(f"MAE per class: {mae_per_class.tolist()}")
    print(f"Overall RMSE: {overall_rmse}")
    print(f"Overall MAE: {overall_mae}")

    cm = confusion_matrix(cls_true_np, cls_pred_np, labels=np.arange(135))

    return {
        "acc": acc,
        "macro_f1": macro_f1,
        "pair_acc": pair_acc,
        "rmse_per_class": rmse_per_class.tolist(),
        "mae_per_class": mae_per_class.tolist(),
        "overall_rmse": overall_rmse,
        "overall_mae": overall_mae,
        "confusion_matrix": cm,
    }

In [ ]:
def create_loaders(root=".", batch_size=64, test_batch_size=1000, device=torch.device("cpu")):
    if device.type == "cuda":
        num_workers = min(8, os.cpu_count() or 2)
    else:
        num_workers = 0

    pin = (device.type == "cuda")
    loader_kwargs = dict(
        num_workers=num_workers,
        pin_memory=pin,
        persistent_workers=False,
    )
    if num_workers > 0:
        loader_kwargs["prefetch_factor"] = 4

    train_aug = Augmentation()
    train_full = GSN(root=root, transform_relabel=train_aug)
    test_full = GSN(root=root)

    train_dataset = Subset(train_full, range(0, 9000))
    test_dataset = Subset(test_full, range(9000, 10000))

    train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True, **loader_kwargs)
    test_loader = DataLoader(test_dataset, batch_size=test_batch_size, shuffle=False, **loader_kwargs)
    return train_loader, test_loader


In [ ]:
epochs = 100
lr = 1e-3
batch_size = 64
test_batch_size = 1000

cls_hidden = 256
cnt_hidden = 256
dropout = 0.3
patience = 10
device = torch.device("cuda" if torch.cuda.is_available() else "mps" if torch.mps.is_available() else "cpu")

def train_model(
    mode: str,
    lambda_cnt = None,
):
    if device.type == "cuda":
        torch.backends.cudnn.conv.fp32_precision = 'tf32'


    train_loader, test_loader = create_loaders(
        root=".",
        batch_size=64,
        test_batch_size=1000,
        device=device,
    )

    net = NeuralNetwork(cls_hidden, cnt_hidden, dropout).to(device)
    optimizer = torch.optim.Adam(net.parameters(), lr=lr)

    history = {
        "train_loss": [],
        "eval_loss": [],
        "eval_acc": [],
        "eval_rmse": [],
    }

    best_eval_loss = float("inf")
    best_eval_loss_acc = 0.0
    best_state = None
    bad_epochs = 0
    best_epoch = 0

    for epoch in range(1, epochs+1):
        train_loss = train_epoch(
            net,
            device,
            train_loader,
            optimizer,
            mode,
            lambda_cnt=lambda_cnt,
        )

        eval_loss, eval_acc, eval_rmse = eval_epoch(
            net,
            device,
            test_loader,
            epoch,
            mode,
            lambda_cnt=lambda_cnt,
            verbose=True
        )

        history["train_loss"].append(train_loss)
        history["eval_loss"].append(eval_loss)
        history["eval_acc"].append(eval_acc)
        history["eval_rmse"].append(eval_rmse)

        if eval_loss < best_eval_loss - 1e-4:
            best_eval_loss = eval_loss
            best_eval_loss_acc = eval_acc
            best_state = {k: v.cpu().clone() for k,v in net.state_dict().items()}
            bad_epochs = 0
            best_epoch = epoch
        else:
            bad_epochs += 1
            if bad_epochs >= patience:
                print(
                    f"Early stop at epoch {epoch}. "
                    f"Best val loss: {best_eval_loss:.4f}, "
                    f"with acc: {best_eval_loss_acc:.4f}, "
                    f"after epoch: {best_epoch}"
                )
                break

    if best_state is not None:
        net.load_state_dict(best_state)

    metrics = evaluate_metrics(net, device, test_loader)

    return net, history, metrics

In [ ]:
results = {}

settings = [
    ("cls_only", None),
    ("reg_only", None),
    ("multitask", 0.3),
    ("multitask", 0.5),
    ("multitask", 1.0),
]

for mode, lambda_cnt in settings:
    if lambda_cnt is not None:
        print(f"\n=== Training mode={mode}, lambda_cnt={lambda_cnt} ===")
        net, history, metrics = train_model(mode=mode, lambda_cnt=lambda_cnt)

        results[(mode, lambda_cnt)] = {
            "history": history,
            "metrics": metrics,
        }
        
    else:
        print(f"\n=== Training mode={mode} ===")
        net, history, metrics = train_model(mode=mode)

        results[mode] = {
            "history": history,
            "metrics": metrics,
        }

In [ ]:
def train_multiple_models(settings, save_dir=None):
    results = {}

    for mode, lambda_cnt in settings:
        if lambda_cnt is not None:
            print(f"\n=== Training mode={mode}, lambda_cnt={lambda_cnt} ===")
            net, history, metrics = train_model(mode=mode, lambda_cnt=lambda_cnt)

            results[(mode, lambda_cnt)] = {
                "history": history,
                "metrics": metrics,
            }

            lambda_str = str(lambda_cnt).replace(".", "_")
            model_path = os.path.join(save_dir, f"model_{mode}_lambda{lambda_str}.pt")
            torch.save(net.state_dict(), model_path)
            print(f"Saved model to {model_path}")
            
        else:
            print(f"\n=== Training mode={mode} ===")
            net, history, metrics = train_model(mode=mode)

            results[mode] = {
                "history": history,
                "metrics": metrics,
            }

            model_path = os.path.join(save_dir, f"model_{mode}.pt")
            torch.save(net.state_dict(), model_path)
            print(f"Saved model to {model_path}")

    results_path = os.path.join(save_dir, "results.pkl")
    with open(results_path, "wb") as f:
        pickle.dump(results, f)
    print(f"\nSaved all results dict to {results_path}")

    return results

In [ ]:
settings = [
    ("cls_only", None),
    ("reg_only", None),
    ("multitask", 0.3),
    ("multitask", 0.5),
    ("multitask", 1.0),
]

In [ ]:
rows = []
for key, res in results.items():
    if isinstance(key, tuple):
        mode, lambda_cnt = key
    else:
        mode, lambda_cnt = key, None

    m = res["metrics"]
    rows.append({
        "mode": mode,
        "lambda_cnt": lambda_cnt,
        "acc": m["acc"],
        "macro_f1": m["macro_f1"],
        "pair_acc": m["pair_acc"],
        "overall_rmse": m["overall_rmse"],
        "overall_mae": m["overall_mae"],
    })

summary_df = pd.DataFrame(rows)
summary_df

In [ ]:
rows = []
index = []
shape_classes = ["square", "circle", "up", "right", "down", "left"]

for key, res in results.items():
    if isinstance(key, tuple):
        mode, lambda_cnt = key
    else:
        mode, lambda_cnt = key, None

    m = res["metrics"]
    rmse_vals = m["rmse_per_class"]
    mae_vals = m["mae_per_class"]

    index.append((mode, lambda_cnt, "rmse"))
    rows.append(rmse_vals)

    index.append((mode, lambda_cnt, "mae"))
    rows.append(mae_vals)

multi_index = pd.MultiIndex.from_tuples(
    index, names=["mode", "lambda_cnt", "metric"]
)

per_class_all_df = pd.DataFrame(rows, index=multi_index, columns=shape_classes)
per_class_all_df

In [ ]:
def plot_confusion_matrix_full(cm, title="Confusion matrix (135 classes)"):
    fig, ax = plt.subplots(figsize=(8, 6))
    im = ax.imshow(cm, interpolation="nearest", aspect="auto")
    fig.colorbar(im, ax=ax)
    ax.set_xlabel("Predicted class")
    ax.set_ylabel("True class")
    ax.set_title(title)
    plt.tight_layout()
    plt.show()

In [ ]:
mt_key = ("multitask", 0.3)
cm = results[mt_key]["metrics"]["confusion_matrix"]
plot_confusion_matrix_full(cm)

In [ ]:
import matplotlib.pyplot as plt

def plot_history(history, mode, title_prefix=""):
    epochs = range(1, len(history["train_loss"]) + 1)

    fig, ax = plt.subplots(figsize=(6, 4))
    ax.plot(epochs, history["train_loss"], label="Train loss")
    ax.plot(epochs, history["eval_loss"], label="Val loss")
    ax.set_xlabel("Epoch")
    ax.set_ylabel("Loss")
    ax.set_title(f"{title_prefix}Loss over epochs")
    ax.legend()
    ax.grid(True)
    fig.tight_layout()
    plt.show()

    if mode != "reg_only":
        fig, ax = plt.subplots(figsize=(6, 4))
        ax.plot(epochs, history["eval_acc"])
        ax.set_xlabel("Epoch")
        ax.set_ylabel("Accuracy")
        ax.set_title(f"{title_prefix}Validation accuracy over epochs")
        ax.grid(True)
        fig.tight_layout()
        plt.show()

    if mode != "cls_only":
        fig, ax = plt.subplots(figsize=(6, 4))
        ax.plot(epochs, history["eval_rmse"])
        ax.set_xlabel("Epoch")
        ax.set_ylabel("RMSE")
        ax.set_title(f"{title_prefix}Validation RMSE over epochs")
        ax.grid(True)
        fig.tight_layout()
        plt.show()


In [ ]:
plot_history(results["reg_only"]["history"], mode="reg_only", title_prefix="[reg_only] ")

In [ ]:
plot_history(results["cls_only"]["history"], mode="cls_only", title_prefix="[cls_only] ")

In [ ]:
plot_history(results[("multitask", 0.3)]["history"], mode="multitask", title_prefix="[multitask λ=0.3] ")

In [ ]:
plot_history(results[("multitask", 0.5)]["history"], mode="multitask", title_prefix="[multitask λ=0.5] ")

In [ ]:
plot_history(results[("multitask", 1.0)]["history"], mode="multitask", title_prefix="[multitask λ=1.0] ")